In [1]:
import lightgbm
print(lightgbm.__version__)

4.6.0


In [2]:
import hyperopt
print(hyperopt.__version__)

0.2.7


In [ ]:
# Hyperopt : ML 모델의 하이퍼파라미터의 최적화(best parameter 찾기) 할때 사용
# 1. 공간(space) 설정 : best parameter가 있을 것 같은 범위 지정
# 2. 평가 지표 설정 : 목적 함수 설정 (사용자가 선언) 보통 오차율
# 3. 최적화 알고리즘(search), TPE(Tree-structured Parzen Estimator) <- 최적의 값을 찾는 알고리즘
# 베이지안 기법(Bayesian Optimization) 중 하나
# 기존의 그리드 탐색(Grid Search)이나 랜덤 탐색(Random Search)보다 더 효율적


#### TPE의 주요 목적

- TPE의 목적은 **하이퍼파라미터 공간**에서 최적의 값을 찾는 것입니다. 여러 번의 실험을 통해 최적화가 이루어지며, 기존의 **그리드 탐색**(Grid Search)이나 **랜덤 탐색**(Random Search)보다 더 효율적입니다.

#### TPE의 작동 원리
- TPE는 **확률적 모델**을 기반으로 하여, 하이퍼파라미터 공간을 효율적으로 탐색합니다. TPE는 두 가지 분포를 모델링하는 방식으로 작동합니다:
    - **좋은 파라미터 분포** (목표 함수가 잘 수행된 파라미터들)
    - **나쁜 파라미터 분포** (목표 함수가 잘 수행되지 않은 파라미터들)

- TPE는 주어진 하이퍼파라미터 공간에서 "좋은" 파라미터를 더 많이 샘플링하려고 합니다. 모델은 하이퍼파라미터 값이 더 잘 성능을 낼 가능성이 있는 부분을 선택하여 점차 최적의 하이퍼파라미터를 찾습니다.

#### TPE의 특징
- **확률적 접근**: TPE는 각 하이퍼파라미터 값이 주는 성능을 확률 모델로 추정합니다. 이를 통해 더 나은 파라미터 조합을 효율적으로 찾을 수 있습니다.
- **대체로 효율적**: 랜덤 탐색보다 빠르고 효율적으로 최적의 하이퍼파라미터를 찾을 수 있습니다.
- **다양한 최적화 문제에 적용 가능**: 머신러닝 모델의 하이퍼파라미터 튜닝뿐만 아니라, 다른 최적화 문제에도 적용할 수 있습니다.

#### TPE 알고리즘의 작동 방식

- TPE는 다음과 같은 방식으로 작동합니다:
    - **모델 초기화**: 파라미터 공간을 정의하고, 해당 공간에서 샘플을 랜덤하게 추출하여 목표 함수를 평가합니다.
    - **확률적 모델 학습**: TPE는 이전 실험 결과를 바탕으로 **좋은 파라미터 분포**와 **나쁜 파라미터 분포**를 학습합니다. 이때, 좋은 파라미터는 목표 함수에서 높은 성능을 보인 값들입니다.
    - **새로운 파라미터 샘플링**: 학습된 분포를 바탕으로 새로운 파라미터를 샘플링하고, 이 샘플에 대해 목표 함수를 다시 평가합니다.
    - **반복**: 이 과정을 반복하면서 점차 좋은 하이퍼파라미터 조합을 찾아냅니다.

In [ ]:
# grid search는 순차적으로 대입해서 찾기 때문에 느리다
# tpe는 공간(구간 or 범위)를 정한 다음 좋은 분포와 나쁜 분포로 구분 짓고
# 나쁜 분포는 버리고 좋은 분포에서만 반복
# 더 빠르게 최적화가 가능하다

In [ ]:
from hyperopt import hp
# 바깥 'x' 는 key, 튜플 안에 있는 'x' 는 접근을 위한 목적 함수
search_space={'x':hp.quniform('x',-10,10,1),
              'y':hp.quniform('y',-15,15,1)
              }
search_space.get('x')

In [4]:
# 목적 함수를 선언, 변숫값과 변수 검색 공간을 가지는 딕셔너리
def objective_func(search_space):
    x=search_space['x']
    y=search_space['y']
    retval=x**2 - 20*y
    
    return retval

# retval = return value

In [5]:
from hyperopt import fmin,tpe,Trials
import numpy as np
import pandas as pd

# 입력 결과값을 저장한 Trials 객체값 생성
trial_val = Trials()

# 목적 함수의 최솟값을 반환하는 최적 입력 변수값을 5번의 입력값 시도(max_evals=5)로 찾아냄
# 목적 함수, 공간을 fmin()에 넣어서 최소가 되는 x,y를 찾는다
best_01=fmin(fn=objective_func,
             space=search_space,
             algo=tpe.suggest,
             max_evals=5,
             trials=trial_val,
             rstate=np.random.default_rng(seed=0)
             )

print(f'best :{best_01}')

100%|██████████| 5/5 [00:00<00:00, 454.47trial/s, best loss: -224.0]
best :{'x': np.float64(-4.0), 'y': np.float64(12.0)}


In [8]:
print(f'best 최솟값: {best_01}')

best 최솟값: {'x': np.float64(-4.0), 'y': np.float64(12.0)}


In [ ]:
# x 가 -4 이고, y 가 12 일때 
((-4.0)**2) - (20 * 12)

-224.0